<a href="https://colab.research.google.com/github/kikiru328/enterprise_analysis/blob/main/ApP_ratio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os, sys
from google.colab import drive
drive.mount('/content/drive')
my_path = '/content/notebooks'
os.symlink('/content/drive/My Drive/Colab Notebooks', my_path)
sys.path.insert(0,my_path)


Mounted at /content/drive


In [2]:
#  Directory 변경 (모듈이용)
%pwd
%cd '../content/notebooks'

# module import
## Data Load
from pickle5 import pickle 

## Basic module
import pandas as pd
import numpy as np
import datetime as dt
import seaborn as sns
import matplotlib.pyplot as plt
plt.rc('font', family='NanumBarunGothic')

import warnings
warnings.filterwarnings('ignore')

/content/drive/My Drive/Colab Notebooks


In [34]:
with open('/content/drive/MyDrive/공유문서/1차 프로젝트/5조/data/EDA/기존_purprd_cust.pkl','rb') as file:
  df = pickle.load(file)


In [5]:
df_1 = df[(df['구매년도']==2014)&(df['구매월']>=1) & (df['구매월']<=6)] 
df_2= df[(df['구매년도']==2014)&(df['구매월']>=7) & (df['구매월']<=12)]
df_3= df[(df['구매년도']==2015)&(df['구매월']>=1) & (df['구매월']<=6)]
df_4 = df[(df['구매년도']==2015)&(df['구매월']>=7) & (df['구매월']<=12)]

In [50]:
df_1 = df[(df['구매년도']==2014) & (df['구매월']>=1) & (df['구매월']<=6)]
df_2 = df[(df['구매년도']==2014)&(df['구매월']>=7) & (df['구매월']<=12)]
df_3 = df[(df['구매년도']==2015)&(df['구매월']>=1) & (df['구매월']<=6)]
df_4 = df[(df['구매년도']==2015)&(df['구매월']>=7) & (df['구매월']<=12)]


def train_test_df(df):
  def countpur(df,a):
    countpur = df[['고객번호','영수증번호','대분류명','구매금액']]
    countpur = countpur.groupby(['고객번호','영수증번호','대분류명']).sum()
    countpur = countpur.reset_index()
    countpur = countpur.groupby(['영수증번호','고객번호']).sum()
    countpur = countpur.reset_index()
    countpur['거래횟수'] = 1 
    countpur = countpur.groupby('고객번호').sum()
    countpur = countpur.drop('영수증번호',axis=1)
    countpur['객단가'] = countpur['구매금액'] / countpur['거래횟수']
    countpur = countpur.drop(['구매금액','거래횟수'],axis=1)
    cols = list(countpur.columns)
    for i in cols:
      countpur[f'{a}_{i}'] = countpur[i]
    countpur = countpur.drop(cols,axis=1)
    return countpur

  H1 = countpur(df_1,'H1')
  H2 = countpur(df_2,'H2')
  H3 = countpur(df_3,'H3')
  H4 = countpur(df_4,'H4') 

  def train_df(a,b,c):
    A = pd.concat([a,b],axis=1)
    train = pd.concat([A,c],axis=1)
    return train

  train = train_df(H1,H2,H3)
  return train, H4
test = H4.copy()

In [51]:
train_test_df(df)
display(train.head())
display(test.head())

,H1_ApP,H2_ApP,H3_ApP
고객번호,,,
1,125316.698113,155782.246575,96747.918919
2,129378.882979,131270.059880,167419.337349
3,6650.117188,6205.678571,6300.851064
4,36803.693431,40547.714286,27290.574627
5,24818.863636,29676.400000,36233.703704


,H4_ApP
고객번호,
1,111038.288136
2,166734.322581
3,6791.180000
4,23804.184713
5,52325.200000


In [54]:
with open('/content/drive/MyDrive/공유문서/1차 프로젝트/5조/진짜진짜최종/123(train)/객단가123.pkl', 'wb') as train_save:
  pickle.dump(train,train_save)
with open('/content/drive/MyDrive/공유문서/1차 프로젝트/5조/진짜진짜최종/4(test)/객단가4.pkl', 'wb') as test_save:
  pickle.dump(test,test_save)